<a href="https://colab.research.google.com/github/zahraniayudyaa/TUGAS-ML-DL/blob/main/ch_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 11: Training Deep Neural Networks**

# **1. MASALAH VANISHING/EXPLODING GRADIENTS**
---
Pada jaringan saraf dalam (deep neural networks), selama proses pelatihan dengan backpropagation, gradien dari fungsi loss dapat menjadi sangat kecil (vanishing gradients) atau sangat besar (exploding gradients) saat melewati banyak lapisan. Hal ini terjadi karena perkalian berulang dari gradien melalui setiap lapisan.

Penyebab utama:
* Fungsi aktivasi yang jenuh seperti sigmoid dan tanh memiliki turunan yang mendekati 0 untuk input besar
* Inisialisasi bobot yang tidak tepat menyebabkan varians input/output tidak stabil
* Jaringan yang sangat dalam memperbesar efek perkalian gradien

Dampak:
* Lapisan bawah (dekat input) tidak mendapatkan update yang berarti
* Pelatihan menjadi sangat lambat atau tidak konvergen
* Model tidak dapat belajar pola kompleks

### Demonstrasi Vanishing Gradients dengan Sigmoid

In [ ]:
# Fungsi sigmoid dan turunannya
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

# Simulasi gradien melalui 10 layer
np.random.seed(42)
gradient = 1.0
for i in range(10):
    weights = np.random.randn(100, 100) * 1.0  # Inisialisasi buruk
    activation_input = np.random.randn(100)
    activation_output = sigmoid(np.dot(weights, activation_input))
    gradient = gradient * sigmoid_derivative(np.dot(weights, activation_input)).mean()
    print(f"Layer {i+1}: Gradient magnitude = {gradient:.10f}")

# **2. TEKNIK INISIALISASI**
---
Inisialisasi bobot yang tepat sangat penting untuk memastikan sinyal dapat merambat dengan baik baik maju (forward) maupun mundur (backward).

**Glorot/Xavier Initialization:**
* Dirancang untuk fungsi aktivasi sigmoid/tanh
* Mempertahankan varians input dan output yang sama

Rumus:
* Distribusi normal: σ² = 1 / fan_avg
* Distribusi uniform: [-√(3/fan_avg), √(3/fan_avg)]

**He Initialization:**
* Dirancang untuk ReLU dan variannya
* Memperhitungkan sifat ReLU yang membuang setengah aktivasi
* Rumus: σ² = 2 / fan_in

**LeCun Initialization:**
* Versi awal dari Glorot initialization
* Efektif untuk fungsi aktivasi SELU

### Implementasi Inisialisasi di Keras

In [ ]:
# Glorot Initialization (default untuk dense layer)
layer_glorot = keras.layers.Dense(100, activation='tanh',
                                 kernel_initializer='glorot_normal')

# He Initialization untuk ReLU
layer_he = keras.layers.Dense(100, activation='relu',
                             kernel_initializer='he_normal')

# LeCun Initialization untuk SELU
layer_lecun = keras.layers.Dense(100, activation='selu',
                                kernel_initializer='lecun_normal')

# Custom VarianceScaling
custom_init = keras.initializers.VarianceScaling(
    scale=2.0, mode='fan_avg', distribution='uniform'
)
layer_custom = keras.layers.Dense(100, activation='sigmoid',
                                 kernel_initializer=custom_init)

# **3. FUNGSI AKTIVASI NON-SATURATING**
---
Fungsi aktivasi tradisional (sigmoid, tanh) mengalami saturasi yang menyebabkan vanishing gradients. Fungsi aktivasi modern dirancang untuk mengatasi masalah ini.

Perbandingan Fungsi Aktivasi:
### **1. ReLU (Rectified Linear Unit):**
* f(x) = max(0, x)
* Kelebihan: Tidak jenuh untuk x > 0, komputasi cepat
* Masalah: Dying ReLU (neuron mati jika selalu output 0)

### **2. Leaky ReLU:**
* f(x) = max(αx, x) dengan α kecil (biasanya 0.01)
* Mengatasi dying ReLU dengan slope kecil untuk x < 0
* ELU (Exponential Linear Unit):
* f(x) = x jika x ≥ 0, α(exp(x)-1) jika x < 0
* Mean output mendekati 0, membantu konvergensi
* Smooth di semua titik (jika α=1)

### **3. SELU (Scaled ELU):**
* ELU yang diskalakan dengan parameter λ dan α
* Dapat menyebabkan self-normalization
* Syarat: arsitektur sequential, input distandardisasi

### Implementasi Fungsi Aktivasi

In [ ]:
# Model dengan berbagai fungsi aktivasi
model_activations = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),

    # ReLU dengan He initialization
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.Activation("relu"),

    # Leaky ReLU
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),

    # SELU dengan LeCun initialization
    keras.layers.Dense(100, activation="selu",
                      kernel_initializer="lecun_normal"),

    # Output layer
    keras.layers.Dense(10, activation="softmax")
])

# **4. BATCH NORMALIZATION**
---
Batch Normalization (BN) menormalkan aktivasi setiap layer untuk mengurangi internal covariate shift. Teknik ini:

* Menstabilkan distribusi input ke setiap layer
* Memungkinkan learning rate lebih besar
* Berfungsi sebagai regularizer ringan
* Mengurangi ketergantungan pada inisialisasi

Algoritma Batch Normalization:
* Hitung mean dan varians mini-batch
* Normalisasi: x̂ = (x - μ) / √(σ² + ε)
* Scale dan shift: y = γ ⊗ x̂ + β

Mode Operasi:
* Training: Gunakan statistik batch
* Inference: Gunakan moving averages

### Implementasi Batch Normalization

In [ ]:
# Model dengan Batch Normalization SETELAH aktivasi
model_bn_after = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(300, activation="elu",
                      kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(100, activation="elu",
                      kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(10, activation="softmax")
])

# Model dengan Batch Normalization SEBELUM aktivasi
model_bn_before = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(300, kernel_initializer="he_normal",
                      use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),

    keras.layers.Dense(100, kernel_initializer="he_normal",
                      use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers